In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import networkx as nx
from nxviz.plots import CircosPlot
import operator
from operator import itemgetter
import pandas as pd
from PIL import Image
import networkx as nx
from matplotlib import pyplot, patches,axes
import matplotlib.pyplot as plt
import os
from collections import defaultdict
from glob import glob
import seaborn as sns

- schaefer200 7 networks label

In [ ]:
label = []
label.append(np.arange(0,14))
label.append(np.arange(14,30))
label.append(np.arange(30,43))
label.append(np.arange(43,54))
label.append(np.arange(54,60))
label.append(np.arange(60,73))
label.append(np.arange(73,100))
label.append(np.arange(100,115))
label.append(np.arange(115,134))
label.append(np.arange(134,147))
label.append(np.arange(147,158))
label.append(np.arange(158,164))
label.append(np.arange(164,181))
label.append(np.arange(181,200))


## load fdr_matrix

In [ ]:
fdr_matrix = np.load('fdr_matrix_normal.npy')

## Rearrange into 7 network (schaefer200)

In [ ]:
Meanvalue_list_nor =[]
num=[]
mean_nor = fdr_matrix.copy()
for row in range(0,14):
    for col in range(0,14):
        sumvalue = fdr_matrix[:,label[col]][label[row],:].mean()
        mean_nor[label[col][0]:label[col][-1]+1,label[row][0]:label[row][-1]+1] = sumvalue
        Meanvalue_list_nor.append(sumvalue)

In [ ]:
mean_14 = []
for i in range (0,14):
    mean_14.append(Meanvalue_list_nor[i*14:i*14+14])

In [ ]:
plt.matshow(mean_14)
plt.colorbar()

In [ ]:
mean_14_abs = np.abs(mean_14)
plt.matshow(mean_14_abs,cmap='Reds')
cb = plt.colorbar()
cb.set_ticks([])
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

### 14x14 to 7x7

In [ ]:
plt.matshow(mean_14_abs,cmap='Reds')
cb = plt.colorbar()
cb.set_ticks([])
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

In [ ]:
plt.matshow(IG_red ,cmap='Reds')
cb = plt.colorbar()
cb.set_ticks([])
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

## Spider/circos plot

### spider plot

In [ ]:
e_weight_mean = []
for i in range(0,7):
    e_weight_mean.append([x+y for x,y in zip (mean_14[i][i+1:7],mean_14[i+7][i+8:14])])

In [ ]:
from math import pi
 
# Set data
df_w = pd.DataFrame({
'group': ['A'],
'Visual': [(mean_14[0][0]+mean_14[7][7])/2],
'Somat': [(mean_14[1][1]+mean_14[8][8])/2],
'Dorsal': [(mean_14[2][2]+mean_14[9][9])/2],
'Ventral': [(mean_14[3][3]+mean_14[10][10])/2],
'Limbic': [(mean_14[4][4]+mean_14[11][11])/2],
'Fronto': [(mean_14[5][5]+mean_14[12][12])/2],
'Default': [(mean_14[6][6]+mean_14[13][13])/2]
})
 
# number of variable
categories=list(df_w)[1:]
N = len(categories)
 
# We are going to plot the first line of the data frame.
# But we need to repeat the first value to close the circular graph:
values=df_w.loc[0].drop('group').values.flatten().tolist()
values += values[:1]
values
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# Draw one axe per variable + add labels
plt.xticks(angles[:-1], categories, color='grey', size=10)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([0,0.75,1.5], ["0","0.75","1.5"], color="grey", size=10)
plt.ylim(0,1.5)
 
# Plot data
ax.plot(angles,np.abs(values), linewidth=2.3, linestyle='solid')

# Fill area
#ax.fill(angles,np.abs(values), 'b', alpha=0.1)

# Show the graph
plt.show()

### Circos plot

In [ ]:
G_node = np.array(df_w.columns[1:])
G = nx.Graph()
G.add_nodes_from(np.array([0,1,2,3,4,5,6]))

In [ ]:
edges=[]
for i in range(0,6):
    for j in range(1+i,7):
        print((G_node[i],G_node[j]),e_weight_mean[i][j-(i+1)])

In [ ]:
numnode = np.array([0,1,2,3,4,5,6])

In [ ]:
for i in range(0,6):
    for j in range(i+1,7):
        G.add_edge(numnode[i],numnode[j],weight = abs(e_weight_mean[i][j-(i+1)]/2))     

In [ ]:
width_weight = nx.get_edge_attributes(G,'weight')

In [ ]:
# Function to transform our list of brain areas into a dictionary
def Convert(lst): 
    res_dct = {i : lst[i] for i in range(0, len(lst))} 
    return res_dct
#linelist 지정
lineList = ['Visual', 'Somat', 'Dorsal', 'Ventral', 'Limbic', 'Fronto', 'Default']

# Obtaining a random list of numbers to simulate subnetworks -- THESE NUMBERS DO NOT CORRESPOND TO ANY REAL CLASSIFICATION
sublist = ['Visual', 'Somatomotor', 'Dorsal attention', 'Ventral attention', 'Limbic', 'Frontoparietal', 'Default mode']

# Obtaining a random list of colors that will match the random subnetwork classification for further graphs -- THESE COLORNAMES DO NOT CORRESPOND TO ANY REAL CLASSIFICATION
colorlist = ['purple','blue','green','violet','cream','orange','red']
color_hex = ['800080','0000FF','008000','EE82EE','FFFDD0','FFA500','FF0000']
colornumbs = np.array([1,2,3,4,5,6,7])

nodesize=np.array([1,1,1,1,1,1,1])

In [ ]:
nx.set_node_attributes(G, dict(G.degree(weight='weight')), 'strength')
nx.set_node_attributes(G, Convert(lineList), 'area')
nx.set_node_attributes(G, Convert(colorlist), 'colorlist')
nx.set_node_attributes(G, Convert(color_hex), 'colorcode')
nx.set_node_attributes(G, Convert(sublist), 'subnet')
# Add node color numbers
nx.set_node_attributes(G, Convert(colornumbs), 'colornumb')
nx.set_node_attributes(G, Convert(nodesize), 'nodesize')
nx.set_edge_attributes(G,G_edge_degree,'edge_width' )

In [ ]:
G_distance_dict2 = {(e1, e2): 1 / abs(weight) for e1, e2, weight in G.edges(data='weight')}

In [ ]:
# Computation of nodal degree/strength

strength = G.degree(weight='weight')
strengths = {node: val for (node, val) in strength}
nx.set_node_attributes(G, dict(strength), 'strength') # Add as nodal attribute

# Normalized node strength values 1/N-1
normstrenghts = {node: val * 1/(len(G.nodes)-1) for (node, val) in strength}
nx.set_node_attributes(G, normstrenghts, 'strengthnorm') # Add as nodal attribute

# Computing the mean degree of the network
normstrengthlist = np.array([val * 1/(len(G.nodes)-1) for (node, val) in strength])

In [ ]:
G_distance_dict2_ = {(e1, e2): 1 / abs(weight) for e1, e2, weight in G.edges(data='weight')}

# Then add them as attributes to the graph
nx.set_edge_attributes(G, G_distance_dict2_, 'distance')

st50GRL = nx.relabel_nodes(G, {i: lineList[i] for i in range(len(lineList))})

# CircosPlot
circ = CircosPlot(st50GRL, figsize=(15,15), node_labels=False,
                   
                   edge_color = 'weight',edge_width='edge_width',node_label_layout='rotation' ,
                  node_color='colorlist', node_label_color=True, fontsize=20, 
                  nodeprops={"radius": 0.0015}, group_legend=True, group_label_offset=5)

circ.draw()

circ.sm.colorbar.remove()
labels_networks = list(set([list(circ.graph.nodes.values())[n][circ.node_color] 
                                  for n in np.arange(len(circ.nodes))]))
labels_networks_ = [list(circ.graph.nodes.values())[n][circ.node_color] for n in np.arange(len(circ.nodes))]

#labels_networks.reverse()
#list(reversed(circ.legend_handles)

plt.legend(handles=circ.legend_handles,
   
           ncol=1,
           borderpad=1,
           shadow=True,
           fancybox=True,
           loc='best',
           fontsize=10,
           labels= sublist)

plt.legend('')
plt.show()

## Drawing IG_matrix 

### asd_IG

In [ ]:
asd_IG= np.load("asd_IG_mean(matrix).npy")

In [ ]:
for i in range(0,200):
    for j in range (0,200):
        if asd_IG[i][j] == 0:
            asd_IG[i][j] = np.nan

In [ ]:
IG_sumvalue_list =[]
IGmean = asd_IG_original.copy()
for row in range(0,14):
    for col in range(0,14):
        IG_sumvalue = asd_IG_original[:,label[col]][label[row],:].mean()
        IGmean[label[col][0]:label[col][-1]+1,label[row][0]:label[row][-1]+1] = IG_sumvalue
        IG_sumvalue_list.append(IG_sumvalue)

In [ ]:
IG_whole_14 = []
for i in range (0,14):
    IG_whole_14.append(IG_sumvalue_list[i*14:i*14+14]) 

In [ ]:
for i in range(0,14):
    for j in range (0,14):
        if IG_whole_14 [i][j] == 0:
            IG_whole_14 [i][j] = np.nan

In [ ]:
IG_seven = np.zeros((7,7))
for i in range(7):
    for j in range(7):
        IG_seven[i][j] = (IG_whole_14[i][j]+IG_whole_14[i+7][j+7])/2

In [ ]:
plt.matshow(IG_seven,vmin = -0.0001 , vmax = 0.0001)
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

### con_IG

In [ ]:
con_IG = np.load("con_IG_mean(matrix).npy")

In [ ]:
IG_sumvalue_list_con =[]
IGmean = con_IG.copy()
for row in range(0,14):
    for col in range(0,14):
        IG_sumvalue = con_IG[:,label[col]][label[row],:].mean()
        IGmean[label[col][0]:label[col][-1]+1,label[row][0]:label[row][-1]+1] = IG_sumvalue
        IG_sumvalue_list_con.append(IG_sumvalue)

In [ ]:
IG_whole_14_con = []
for i in range (0,14):
    IG_whole_14_con.append(IG_sumvalue_list_con[i*14:i*14+14]) 

In [ ]:
for i in range(0,14):
    for j in range (0,14):
        if IG_whole_14_con [i][j] == 0:
            IG_whole_14_con [i][j] = np.nan

In [ ]:
IG_seven_con = np.zeros((7,7))
for i in range(7):
    for j in range(7):
        IG_seven_con[i][j] = (IG_whole_14_con[i][j]+IG_whole_14_con[i+7][j+7])/2

In [ ]:
plt.matshow(IG_seven_con,vmin = -0.0001 , vmax = 0.0001)
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)